In [49]:
import lark

grammar = """
start:      expression

expression: term (("+" | "-") term)*
term:       factor (("*" | "/") factor)*
factor:     ("+" | "-") factor | power
power:      call ["**" factor]
call:       atom trailer*
atom:       "(" expression ")" | CNAME -> symbol | NUMBER -> literal

trailer:    "(" arglist ")"
arglist:    expression ("," expression)*

%import common.CNAME
%import common.NUMBER
%import common.WS

%ignore WS
"""

parser = lark.Lark(grammar)


In [52]:
print(parser.parse("2 + 2").pretty())

start
  expression
    term
      factor
        power
          call
            literal	2
    term
      factor
        power
          call
            literal	2



In [26]:
class AST:
    _fields = ()
    def __init__(self, *args, line=None):
        for n, x in zip(self._fields, args):
            setattr(self, n, x)
        self.line = line
    def __repr__(self):
        return "{0}({1})".format(type(self).__name__, ", ".join(getattr(self, n) for n in self._fields))

class Literal(AST):
    _fields = ("value",)
    def __str__(self):
        return str(self.value)

class Symbol(AST):
    _fields = ("symbol",)
    def __str__(self):
        return self.symbol

class Call(AST):
    _fields = ("function", "arguments")
    def __str__(self):
        return "{0}({1})".format(str(self.function), ", ".join(str(x) for x in self.arguments))

In [58]:
def recurse(parsed):
    if parsed.data == "literal":
        return Literal(float(parsed.children[0]))
    elif parsed.data == "symbol":
        return Symbol(parsed.children[0])
    elif parsed.data == "call":
        return

    if isinstance(parsed, lark.lexer.Token):
        print("token", parsed)

    # for x in parsed.children:
    #     recurse(x)

(parser.parse("2").children[0].children[0].children[0].children[0]).data


'power'